In [1]:
import socket
import tensorflow.keras
import numpy as np
import cv2
import img_prepare
import kakaonotice

## tcp start

In [2]:
HOST = ''
PORT = 4000

In [3]:
#socket 객체
# IPv4, TCP 
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #TCP
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#bind
server_socket.bind((HOST, PORT))

#allow connection
server_socket.listen(5)

#wait on accept func. when connected, return new socket
client_socket, addr = server_socket.accept()
print("Connected by", addr)

Connected by ('192.168.228.98', 8010)


## get tf model 

In [4]:
#레이블 가져오기
labels=[]
f=open("labels.txt", "r")
for x in f:
    labels.append(x.rstrip('\n'))
    label_count = len(labels)
f.close()


# e-04와 같은 scientific notation을 제거하고 싶을 때 사용하는 옵션
np.set_printoptions(suppress=True)

# Teachable Machine에서 학습시킨 모델 파일을 모델 파일을 컴파일을 다시 하지 않고, model 변수에 넣음
model = tensorflow.keras.models.load_model('keras_model.h5', compile=False)


## prepare video

## show predict and call

In [5]:
# numpy를 이용해 이미지를 1차원, 높이 224pixel, 폭 224pixel, 색상 3채널(RGB)로 변환해서 data 변수에 넣음, 형식은 float32, 여기서는 data 변수를 만드는 의미로 쓰임
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# OpenCV를 이용해 캠으로 들어오는 영상을 cap 변수에 넣음, 
# '0'은 컴퓨터가 인식한 첫번째 카메라를 의미함
cap = cv2.VideoCapture(cv2.CAP_DSHOW+1)
#cap = cv2.VideoCapture(0)

predict_thresh = 0.7

while True:
    
    ret, frame = cap.read()

    if not ret:
        print("cam off")
        # OpenCV를 이용해 캠으로 들어오는 영상을 cap 변수에 넣음, '0'은 컴퓨터가 인식한 첫번째 카메라를 의미함
        cap = cv2.VideoCapture(cv2.CAP_DSHOW+1)
        #cap = cv2.VideoCapture(0)
        continue

    crop_frame = img_prepare.img_crop(frame)
    normalized_frame = img_prepare.img_normalize(crop_frame)
    
    # 정규화된 배열을 data[0]에 넣음
    data[0] = normalized_frame

    # 정규화된 배열값으로 정돈된 data를 Teachable Machine으로 학습시켜서 얻은 모델을 이용해 추론하고, 그 결과를 prediction 변수에 넣음
    prediction = model.predict(data)

    # 글씨 넣기 준비
    font = cv2.FONT_HERSHEY_TRIPLEX
    fontScale = 1
    fontColor = (0,255,0)
    lineThickness = 1

    # 표기 문구 초기화
    scoreLabel = 0
    score = 0
    result = ''

    for x in range(0, label_count):
        #예측값 모니터링
        line=('%s=%0.0f' % (labels[x], int(round(prediction[0][x]*100)))) + "%"
        cv2.putText(crop_frame, line, (10,(x+1)*35), font, fontScale, fontColor, lineThickness)

        # 가장 높은 예측 찾기
        if score < prediction[0][x]:
            scoreLabel = labels[x]
            score = prediction[0][x]
            result = str(scoreLabel) + " : " + str(score)
            print(result)

    # 최고 결과치 보여주기
    crop_frame = cv2.putText(crop_frame, result, (10, int(label_count+1)*35), font, 1, (0, 0, 255), 1, cv2.LINE_AA)
    
    # 크롭이미지 화면에 보여주기
    cv2.imshow('crop_image',crop_frame)

    # prediction의 첫번째 리스트값이 predict_thresh 넘으면 'a'를 utf-8 형태로 인코딩하여 시리얼 통신으로 송신함
    # 학습을 시킬 때는 반드시 디폴트 이미지를 학습시키는 것이 오류를 예방할 수 있는 지름길임
    if prediction[:, 0] > predict_thresh :
    # 'a'를 utf-8 형식으로 인코딩 하여 send 변수에 넣음
        send = "Event:1"
    # 송신이 되면 화면에 send 변수에 들어가 있는 값을 출력함
        print(send)

    # # prediction의 두번째 리스트값이 0.7을 넘으면 'b'를 utf-8 형태로 인코딩하여 시리얼 통신으로 송신함
    # elif prediction[:, 1] > predict_thresh :
    #     send = "Event:2"
    #     print(send)

    # # prediction의 세번째 리스트값이 0.7을 넘으면 'c'를 utf-8 형태로 인코딩하여 시리얼 통신으로 송신함
    # elif prediction[:, 2] > predict_thresh :
    #     send = "Event:3"
    #     print(send)

    else:
        #do nothing
        send = "Event:0"
        print(send)

    key = cv2.waitKey(1) & 0xFF

    #클라이언트로부터 수신되는 메세지 처리
    #stop when receive Q
    receive = client_socket.recv(128).decode()
    if receive == 'Q' or receive == 'q':
        client_socket.send(send.encode())
        client_socket.close()

        print('Quit')
        break
    elif receive == "ENQ":
        #print receive data
        client_socket.send(send.encode())
    elif receive == "E1":
        #client event:1 경고 시작
        #code = kakaonotice.pushMSG("집안에서 이상증세가 발생했습니다. Event:1")
        #print("code:",code)
        print("code:1")
    elif receive == "E2":
        #client event:2 경고 시작
        #code = kakaonotice.pushMSG("집안에서 이상증세가 발생했습니다. Event:2")
        #print("code:",code)
        print("code:2")


# 동작이 종료되면 비디오 프레임 캡쳐를 중단함
cap.release()
# 모든 창을 닫음
cv2.destroyAllWindows()
server_socket.close()
print("Socket closed...")

0 event1 : 0.026753647
1 default : 0.9732464
Event:0
0 event1 : 0.025793087
1 default : 0.97420686
Event:0
0 event1 : 0.026048077
1 default : 0.97395194
Event:0
0 event1 : 0.02615506
1 default : 0.97384495
Event:0
0 event1 : 0.001439311
1 default : 0.9985607
Event:0
0 event1 : 0.01273904
1 default : 0.98726094
Event:0
0 event1 : 0.008317532
1 default : 0.9916824
Event:0
0 event1 : 0.041813064
1 default : 0.958187
Event:0
0 event1 : 0.002516433
1 default : 0.99748355
Event:0
0 event1 : 0.010726106
1 default : 0.98927385
Event:0
0 event1 : 0.009301999
1 default : 0.99069804
Event:0
0 event1 : 0.002373166
1 default : 0.99762684
Event:0
0 event1 : 0.0017459008
1 default : 0.9982541
Event:0
0 event1 : 0.0024634607
1 default : 0.99753654
Event:0
0 event1 : 0.0033268046
1 default : 0.9966732
Event:0
0 event1 : 0.0032974978
1 default : 0.9967025
Event:0
0 event1 : 0.0027127385
1 default : 0.9972873
Event:0
0 event1 : 0.0026301376
1 default : 0.9973699
Event:0
0 event1 : 0.0030471063
1 default 

KeyboardInterrupt: 